In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=67bb808fefa8027f450e9b4c07cc29252b967b05b06c4b6a9ce7c7dd21a4a257
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


# Logistic Regression
Let's see an example of how to run a logistic regression with Python and Spark! This is documentation example, we will quickly run through this and then show a more realistic example, afterwards, you will have another consulting project!

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('mylogreg').getOrCreate()

In [4]:
from pyspark.ml.classification import LogisticRegression

In [13]:
# Load training data
my_data=spark.read.format('libsvm').load('/content/sample_data/sample_libsvm_data.txt')
my_data.show()

my_log_reg_model=LogisticRegression()
## Fit the model
fitted_logreg=my_log_reg_model.fit(my_data)

log_summary=fitted_logreg.summary

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [14]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [15]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823766...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446132...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [16]:
lr_train,lr_test=my_data.randomSplit([0.7,0.3])

In [17]:
final_model=LogisticRegression()

In [18]:
fit_final=final_model.fit(lr_train)

In [19]:
prediction_and_labels=fit_final.evaluate(lr_test)

In [20]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[123,124,125...|[35.3351405081332...|[0.99999999999999...|       0.0|
|  0.0|(692,[124,125,126...|[39.1703381705621...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[34.7877265150717...|[0.99999999999999...|       0.0|
|  0.0|(692,[124,125,126...|[26.0028612917426...|[0.99999999999490...|       0.0|
|  0.0|(692,[126,127,128...|[29.3903997584922...|[0.99999999999982...|       0.0|
|  0.0|(692,[126,127,128...|[23.3626482328486...|[0.99999999992859...|       0.0|
|  0.0|(692,[127,128,129...|[18.2787343364501...|[0.99999998847485...|       0.0|
|  0.0|(692,[127,128,129...|[26.0563075408779...|[0.99999999999517...|       0.0|
|  0.0|(692,[128,129,130...|[21.6421180530668...|[0.99999999960102...|       0.0|
|  0.0|(692,[129

In [25]:
# May change soon!
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,MulticlassClassificationEvaluator)

In [26]:
my_eval=BinaryClassificationEvaluator()

In [28]:
my_final_roc=my_eval.evaluate(prediction_and_labels.predictions)

In [29]:
my_final_roc

1.0